In [89]:
import random
import re
import pandas as pd
import numpy as np
import sklearn as sk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
def read():
    return pd.read_csv("dataset/Corona_NLP_train.csv"), pd.read_csv("dataset/Corona_NLP_test.csv")

def remove_urls(text):
    url_remover = re.compile(r'https?://\S+|www\.\S+')
    return url_remover.sub(r'', text)   
    
def remove_html(text):
    html_remover=re.compile(r'<.*?>')
    return html_remover.sub(r'',text)

def remove_mentions(text):
    mention_remover=re.compile(r'@\w+')
    return mention_remover.sub(r'',text)

def remove_numbers(text):
    number_remover=re.compile(r'\d+')
    return number_remover.sub(r'',text)

def remove_hashtags(text):
    number_remover=re.compile(r'#\w+')
    return number_remover.sub(r'',text)

def remove_punctuation(text):
    punct_remover=re.compile(r'[^\w\s\d]+')
    return punct_remover.sub(r'',text)

def remove_excessive_whitespace(text):
    ws_remover=re.compile(r'\s+')
    return ws_remover.sub(r' ', str(text)).strip()

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))     
    return " ".join([word for word in text.split(" ") if word not in stop_words]) 
    
def lowering(text):
    return text.lower()




def clean(df):
    df = df.loc[:,["OriginalTweet", "Sentiment"]]
    scores = {
        'Neutral': 0,     
        'Extremely Negative':-1, 
        'Negative':-1,
        'Positive':1,
        'Extremely Positive':1
    }
    df["score"] = df.Sentiment.apply(lambda x: scores[x])
    df.drop("Sentiment", inplace=True, axis=1)
    df.rename({"OriginalTweet": "text"}, axis=1, inplace=True)
    
    df["text"] = df.text.apply(lambda x: remove_urls(x))
    df["text"] = df.text.apply(lambda x: remove_mentions(x))
    df["text"] = df.text.apply(lambda x: remove_html(x))
    df["text"] = df.text.apply(lambda x: remove_numbers(x))
    df["text"] = df.text.apply(lambda x: remove_hashtags(x))
    df["text"] = df.text.apply(lambda x: remove_punctuation(x))
    df["text"] = df.text.apply(lambda x: remove_excessive_whitespace(x))
    df["text"] = df.text.apply(lambda x: remove_stopwords(x))
    df["text"] = df.text.apply(lambda x: lowering(x))
    
    return df

def main():
    df_train, df_test = read()
    df_train = clean(df_train)
    df_test = clean(df_test)
    
    print(df_train.head())
    print(df_train.tail())
    print(df_train.info())
    print(df_train.describe())
    return df_train, df_test
    

In [106]:
df_train, df_test = main()



                                                text  score
0                                                         0
1  advice talk neighbours family exchange phone n...      1
2  coronavirus australia woolworths give elderly ...      1
3  my food stock one empty please dont panic ther...      1
4  me ready go supermarket outbreak not im parano...     -1
                                                    text  score
41152  airline pilots offering stock supermarket shel...      0
41153  response complaint provided citing covid relat...     -1
41154  you know getting tough rationing toilet paper ...      1
41155        is wrong smell hand sanitizer starting turn      0
41156  well newused rift s going amazon rn although n...     -1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    41157 non-null  object
 1   score   41157 non-null  int64 
dtyp

,text,score
0,,0
1,advice talk neighbours family exchange phone n...,1
2,coronavirus australia woolworths give elderly ...,1
3,my food stock one empty please dont panic ther...,1
4,me ready go supermarket outbreak not im parano...,-1
...,...,...
41152,airline pilots offering stock supermarket shel...,0
41153,response complaint provided citing covid relat...,-1
41154,you know getting tough rationing toilet paper ...,1
41155,is wrong smell hand sanitizer starting turn,0
